In [20]:
import open3d as o3d
print(o3d.__version__)


0.19.0+b53a3ad


In [21]:
try:
    import pyorbbecsdk as ob
    # Most versions expose __version__:
    print("PyOrbbecSDK version:", ob.__version__)
except (ImportError, AttributeError):
    # Fallback: use importlib.metadata (Python 3.8+)
    from importlib.metadata import version, PackageNotFoundError
    try:
        print("PyOrbbecSDK version:", version("pyorbbecsdk"))
    except PackageNotFoundError:
        print("PyOrbbecSDK is not installed.")


PyOrbbecSDK version: 2.0.13


In [22]:
from pyorbbecsdk import Pipeline, Config, OBSensorType, OBFormat

In [23]:
import numpy as np

In [24]:
import cv2

In [25]:


def test_orbbec_depth_only(timeout_ms=2000):
    # 1) Set up Config & Pipeline
    cfg = Config()
    pipe = Pipeline()

    # 2) Pick the first available depth profile
    depth_profiles = pipe.get_stream_profile_list(OBSensorType.DEPTH_SENSOR)
    if not depth_profiles:
        print("No depth sensor found!")
        return
    cfg.enable_stream(depth_profiles[0])  # default depth stream :contentReference[oaicite:0]{index=0}

    # 3) Start streaming
    pipe.start(cfg)

    # 4) Wait for one frameset
    frames = pipe.wait_for_frames(timeout_ms)
    if frames is None:
        print(f"No frame received within {timeout_ms} ms")
        pipe.stop()
        return

    # 5) Extract depth data
    depth_frame = frames.get_depth_frame()
    h, w = depth_frame.get_height(), depth_frame.get_width()
    raw = depth_frame.get_data()
    # ensure we have a bytes object
    buf = raw.tobytes() if hasattr(raw, "tobytes") else bytes(raw)
    arr = np.frombuffer(buf, dtype=np.uint16)
    depth = arr.reshape((h, w))

    # 6) Inspect
    print(f"Depth frame shape: {depth.shape}")
    print(f"Center pixel depth: {depth[h//2, w//2]} mm")

    # 7) Clean up
    pipe.stop()

if __name__ == "__main__":
    test_orbbec_depth_only()


Depth frame shape: (576, 640)
Center pixel depth: 0 mm
